In [ ]:
#check version of tensorflow
import random as rn
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import os
import glob
import datetime
import matplotlib.pyplot as plt
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable
import bindat

In [ ]:
BTC1d = pd.read_csv("F:\\Thesis\\covid-nift\\Dataset\\BTC1d.csv")
BTC1h = pd.read_csv("F:\\Thesis\\covid-nift\\Dataset\\BTC1h.csv")
BTC15m = pd.read_csv("F:\\Thesis\\covid-nift\\Dataset\\BTC15m.csv")
BTC = pd.read_csv("F:\\Thesis\\covid-nift\\Dataset\\BTC1m.csv")
BTC = BTC.set_index(BTC['Date'])
BTC = BTC.drop(['Date'], axis=1)
BTC.index = pd.to_datetime(BTC.index)


--

In [128]:
def split_sequences(sequences, n_steps_in, n_steps_out):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps_in
            out_end_ix = end_ix + n_steps_out-1
            # check if we are beyond the dataset
            if out_end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix-1:out_end_ix,-1]
            X.append(seq_x)
            y.append(seq_y)
        return np.array(X), np.array(y)

In [129]:
class split:

    def __init__(self, df, break_date,time_step):
       
        self.df = df
        self.break_date = break_date
        self.time_step = time_step
        
        #breaking the data into train and test along time dim
        self.test_percent = 0.20
        self.no_test_obs =  int(np.round(self.test_percent*len(self.df)))
        self.training = self.df[:-self.no_test_obs]
        self.testing = self.df[-self.no_test_obs:]

        self.validation = self.testing.loc[self.testing.index <= self.break_date]
        self.outofsample = self.testing.loc[self.testing.index > self.break_date]
        
        #size
        self.train_size = len(self.training)
        self.test_size = len(self.testing)
        
        #minmax
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.trainMinmax = self.scaler.fit_transform(self.training.values) #fit and transform training data
        self.validationMinmax = self.scaler.transform(self.validation.values)
        self.outofsampleMinmax = self.scaler.transform(self.outofsample.values)
        
        #split_sequence
        self.x_seq_train, self.y_seq_train = split_sequences(self.trainMinmax,self.time_step, 1)
        self.x_seq_validation, self.y_seq_validation = split_sequences(self.validationMinmax, self.time_step, 1)
        self.x_seq_outofsample, self.y_seq_outofsample = split_sequences(self.outofsampleMinmax, self.time_step, 1)
              
        # make training and test sets in torch
        self.x_train = torch.from_numpy(self.x_seq_train).type(torch.Tensor)
        self.x_validation = torch.from_numpy(self.x_seq_validation).type(torch.Tensor)
        self.x_outofsample = torch.from_numpy(self.x_seq_outofsample).type(torch.Tensor)

        self.y_train = torch.from_numpy(self.y_seq_train).type(torch.Tensor)
        self.y_validation = torch.from_numpy(self.y_seq_validation).type(torch.Tensor)
        self.y_outofsample = torch.from_numpy(self.y_seq_outofsample).type(torch.Tensor)
    
    def get_train_set(self):
        return self.x_train , self.y_train
    
    def get_validation_set(self):
        return self.x_validation , self.y_validation
    
    def get_outofsample_set(self):
        return self.x_outofsample , self.y_outofsample
    
    def get_test(self):
        return self.validation , self.outofsample
    
    
    def print_split(self):
        print('x train = ',self.x_train.shape)
        print('y train = ',self.y_train.shape)
        print('x Validation = ',self.x_validation.shape)
        print('y Validation = ',self.y_validation.shape)
        print('x outSample = ',self.x_outofsample.shape)
        print('y outSample = ',self.y_outofsample.shape)
        #print('minmax ', self.trainMinmax)
        

In [130]:

# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim,  output_dim, num_layers):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach the initi 
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

       #reshaping the data for Dense layer next
        out = self.fc(out[:, -1, :]) 
        
        return out


In [131]:
 def plot_results(actual_y,predicted_y,method,date):
        mse = mean_squared_error(actual_y,predicted_y)

        plt.figure(figsize=(16,4))
        plt.plot(date,actual_y)
        plt.plot(date,predicted_y)
        plt.legend(['Actual','Predicted'])
        plt.title(f'{method} (MSE: {mse})')        
        plt.show()

In [132]:
class Trainer:

    def __init__(self, df, break_date, time_step, num_epochs, input_dim, hidden_dim, num_layers, output_dim ):
        
        self.df = df 
        self.num_epochs=num_epochs
        self.hist= np.zeros(self.num_epochs)
        self.input_dim=input_dim
        self.hidden_dim=hidden_dim 
        self.num_layers=num_layers 
        self.output_dim=output_dim
        self.time_step = time_step
        self.break_date = break_date
        self.split = split(self.df,self.break_date,self.time_step)
        self.scaler = self.split.scaler
        self.x_train , self.y_train = self.split.get_train_set()
        self.x_validation , self.y_validation = self.split.get_validation_set()
        
    def train_pred(self):
        
        self.model = LSTM(1, 32, 1, 2)
        self.loss_fn = torch.nn.MSELoss()
        self.hist = np.zeros(self.num_epochs)
        self.optimiser = torch.optim.Adam(self.model.parameters(), lr=0.01)
        
        for t in range(self.num_epochs):
            
            # Forward pass
            self.train_prediction = self.model(self.x_train)
            
            self.loss= self.train_iteration(self.train_prediction, self.y_train)
            if t % 10 == 0 and t !=0:
                print("Epoch ", t, "MSE: ", self.loss.item())
            self.hist[t] = self.loss.item()
        plt.plot(self.hist, label="Training loss")
        plt.legend()
        plt.show()
        
        # make predictions
        self.validation_prediction = self.model(self.x_validation)
        
        # invert predictions
        self.train_prediction = self.scaler.inverse_transform(self.train_prediction.detach().numpy())
        self.y_train = self.scaler.inverse_transform(self.y_train.detach().numpy())
        self.validation_prediction = self.scaler.inverse_transform(self.validation_prediction.detach().numpy())
        self.y_validation = self.scaler.inverse_transform(self.y_validation.detach().numpy())
        
        plot_results(self.y_train ,self.train_prediction  ,'LSTM train',range(len(self.y_train )))
        plot_results(self.y_validation ,self.validation_prediction  ,'LSTM Testing',range(len(self.y_validation)))

            
    def train_iteration(self, pred, true):
       # Zero out gradient, else they will accumulate between epochs
        self.optimiser.zero_grad()

        self.loss = self.loss_fn(pred, true)
           
        # Backward pass
        self.loss.backward()

        # Update parameters
        self.optimiser.step()
        return self.loss
    
    def count_error(self, train_prediction,test_prediction,y_train,y_test):
        # calculate root mean squared error
        self.trainScore = math.sqrt(mean_squared_error(self.y_train[:,0], self.train_prediction[:,0]))
        print('Train Score: %.4f RMSE' % (self.trainScore))
        self.validationScore = math.sqrt(mean_squared_error(self.y_validation[:,0], self.validation_prediction[:,0]))
        print('Test Score: %.4f RMSE' % (self.validationScore))
    
    def collect_data(self):
        return self.train_prediction, self.y_train, self.validation_prediction, self.y_validation
        

In [ ]:
p= split(BTC,break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1)
p.print_split()

In [ ]:
#open data
df = BTC.filter(items= ['Open'])
trainer = Trainer(df,break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1,num_epochs = 500,input_dim = 1,
                  hidden_dim = 32, num_layers = 2, output_dim = 1)
trainer.train_pred()
train_prediction, y_train, validation_prediction, y_validation  = trainer.collect_data()
trainer.count_error(train_prediction, y_train, validation_prediction, y_validation)


In [ ]:
s = split(df, break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1)
x_outofsample, y_outofsample = s.get_outofsample_set()
validation, outofsample = s.get_test()

model = LSTM(1, 32, 1, 2)
outofsample_prediction = model(x_outofsample)

scaler = s.scaler
outofsample_prediction = scaler.inverse_transform(outofsample_prediction.detach().numpy())
y_outofsample = scaler.inverse_transform(y_outofsample.detach().numpy())

testScore = math.sqrt(mean_squared_error(y_outofsample[:,0], outofsample_prediction[:,0]))
print('Test Score: %.4f RMSE' % (testScore))


In [ ]:
compare = outofsample[(1-1):]
compare['Predicted_Open'] = outofsample_prediction
compare

In [ ]:
#Close data
df = BTC.filter(items= ['High'])
trainer = Trainer(df,break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1,num_epochs = 500,input_dim = 1,
                  hidden_dim = 32, num_layers = 2, output_dim = 1)
trainer.train_pred()
train_prediction, y_train, validation_prediction, y_validation  = trainer.collect_data()
trainer.count_error(train_prediction, y_train, validation_prediction, y_validation)


Epoch  10 MSE:  0.1919272243976593
Epoch  20 MSE:  0.11204919964075089
Epoch  30 MSE:  0.031905341893434525
Epoch  40 MSE:  0.012045836076140404
Epoch  50 MSE:  0.0064510321244597435
Epoch  60 MSE:  0.002612245036289096
Epoch  70 MSE:  0.0006699168588966131
Epoch  80 MSE:  0.0007400282192975283
Epoch  90 MSE:  0.0006880550063215196
Epoch  100 MSE:  0.0005709797260351479
Epoch  110 MSE:  0.0005135405226610601
Epoch  120 MSE:  0.0004746796330437064
Epoch  130 MSE:  0.0004383159102872014
Epoch  140 MSE:  0.0004054248856846243
Epoch  150 MSE:  0.00037443373003043234
Epoch  160 MSE:  0.0003444950270932168
Epoch  170 MSE:  0.0003157443134114146
Epoch  180 MSE:  0.0002881737018469721
Epoch  190 MSE:  0.00026172780781053007
Epoch  200 MSE:  0.00023641559528186917
Epoch  210 MSE:  0.00021224742522463202
Epoch  220 MSE:  0.00018925276526715606
Epoch  230 MSE:  0.00016747950576245785
Epoch  240 MSE:  0.0001469882990932092
Epoch  250 MSE:  0.0001278497657040134
Epoch  260 MSE:  0.00011013828043360

In [ ]:
s = split(df, break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1)
x_outofsample, y_outofsample = s.get_outofsample_set()

model = LSTM(1, 32, 1, 2)
outofsample_prediction = model(x_outofsample)

scaler = s.scaler
outofsample_prediction = scaler.inverse_transform(outofsample_prediction.detach().numpy())
y_outofsample = scaler.inverse_transform(y_outofsample.detach().numpy())

testScore = math.sqrt(mean_squared_error(y_outofsample[:,0], outofsample_prediction[:,0]))
print('Test Score: %.4f RMSE' % (testScore))

In [ ]:
compare['Predicted_High'] = outofsample_prediction

In [ ]:
#close data
df = BTC.filter(items= ['Close'])
trainer = Trainer(df,break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1,num_epochs = 500,input_dim = 1,
                  hidden_dim = 32, num_layers = 2, output_dim = 1)
trainer.train_pred()
train_prediction, y_train, validation_prediction, y_validation  = trainer.collect_data()
trainer.count_error(train_prediction, y_train, validation_prediction, y_validation)

In [ ]:
s = split(df, break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1)
x_outofsample, y_outofsample = s.get_outofsample_set()

model = LSTM(1, 32, 1, 2)
outofsample_prediction = model(x_outofsample)

scaler = s.scaler
outofsample_prediction = scaler.inverse_transform(outofsample_prediction.detach().numpy())
y_outofsample = scaler.inverse_transform(y_outofsample.detach().numpy())

testScore = math.sqrt(mean_squared_error(y_outofsample[:,0], outofsample_prediction[:,0]))
print('Test Score: %.4f RMSE' % (testScore))

In [ ]:
compare['Predicted_Close'] = outofsample_prediction

In [ ]:
#Low data
df = BTC.filter(items= ['Low'])
trainer = Trainer(df,break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1,num_epochs = 500,input_dim = 1,
                  hidden_dim = 32, num_layers = 2, output_dim = 1)
trainer.train_pred()
train_prediction, y_train, validation_prediction, y_validation  = trainer.collect_data()
trainer.count_error(train_prediction, y_train, validation_prediction, y_validation)

In [ ]:
s = split(df, break_date =  pd.Timestamp("2021-04-24 23:59:00"),time_step = 1)
x_outofsample, y_outofsample = s.get_outofsample_set()

model = LSTM(1, 32, 1, 2)
outofsample_prediction = model(x_outofsample)

scaler = s.scaler
outofsample_prediction = scaler.inverse_transform(outofsample_prediction.detach().numpy())
y_outofsample = scaler.inverse_transform(y_outofsample.detach().numpy())

testScore = math.sqrt(mean_squared_error(y_outofsample[:,0], outofsample_prediction[:,0]))
print('Test Score: %.4f RMSE' % (testScore))

In [ ]:
compare['Predicted_Low'] = outofsample_prediction
sub = compare['Open']-compare['Predicted_Open']
sub

In [ ]:
compare

In [ ]:
i="1min"
compare = compare.drop(["Open"], axis=1)
path = 'F:/Thesis/covid-nift/Dataset/Prediction_files/'
compare.to_csv (path+i+'.csv') 
compare